In [1]:

using Pkg
Pkg.activate("./baron_env")
#Pkg.pin("JuMP")
using JuMP
using CSV
using GraphRecipes, Plots
using LightGraphs
using LinearAlgebra
using SparseArrays
using DataFrames
using BARON
using PiecewiseLinearOpt


  Activating environment at `~/Documents/project-codes/Plan_RoadmapsIAM/toy-problem/baron_env/Project.toml`


In [2]:
Pkg.installed()

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Pkg/src/Pkg.jl:570


Dict{String, VersionNumber} with 3 entries:
  "BARON"              => v"0.6.5"
  "PiecewiseLinearOpt" => v"0.3.0"
  "CPLEX"              => v"0.7.8"

In [30]:
innovations = CSV.File("./data/innovations.csv",header=1,delim=",") |> DataFrame
mapki = CSV.File("./data/map-ki.csv",header=1,delim=",") |> DataFrame
compromise = CSV.File("./data/compromise.csv",header=1,delim=",") |> DataFrame
initial = CSV.File("./data/initial.csv",header=1,delim=",") |> DataFrame
iam_model = CSV.File("./data/iam_model.csv",header=1,delim=",") |> DataFrame
ci=innovations[!,2]
rl=innovations[!,3]
initial

#innovations[2,6]=7

#compromise[!,3]=compromise[!,3]/2
#innovations[2,2]=175
#innovations[2,3]=2
#innovations[1,3]=3
#innovations[4,2]=108

#innovations[1,2]=185
#innovations[3,3]=2
innovations
#CSV.write("innovations.csv",innovations)

,i,Cost_per_RL,RL,Sojourn1,Sojourn2,Sojourn3,Sojourn4,Sojourn5,P11
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Int64
1,1,92,3,4.6,1.34,2.04,0.12,8.86,0
2,2,175,2,8.32,8.84,0.65,9.9,9.65,0
3,3,78,1,4.44,1.21,5.09,6.52,8.67,0
4,4,108,3,5.48,7.62,3.34,5.05,5.6,0
5,5,95,1,9.94,2.38,7.94,3.82,5.44,0


In [26]:
compromise

,k,i,GWP,Theta,Per_Recycl,Per_Comp,GHG_ElecInd,GHG_Trans
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,40.0,0.6,0.5,0.0,0.301222,0.162415
2,2,1,43.0,0.66,0.55,0.0,0.291677,0.145083
3,3,1,45.5,0.68,0.567,0.0,0.323371,0.161281
4,4,1,47.5,0.7,0.583,0.0,0.525025,0.151392
5,5,2,30.0,0.4,0.0,0.2,0.308217,0.0923318
6,6,2,32.0,0.48,0.0,0.24,0.419839,0.167281
7,7,2,34.0,0.5,0.0,0.27,0.418893,0.136562
8,8,2,38.0,0.52,0.0,0.34,0.530551,0.134121
9,9,2,40.0,0.54,0.0,0.37,0.418593,0.146208


In [5]:
function get_transition_mat(i)
    p_mat=reshape(collect(innovations[i,9:33]),5,5)
    return p_mat'
end


function get_rate_mat(i)
    p_mat=get_transition_mat(i)
    q_mat=p_mat./collect(innovations[i,4:8])
    for i in 1:5
        q_mat[i,i]=-1*sum(q_mat[i,j] for j in 1:5 if j!=i)
    end
    return q_mat
end

function get_rate_eig_mat(i)
    p_mat=get_transition_mat(i)
    q_mat=p_mat./collect(innovations[i,4:8])
    for i in 1:5
        q_mat[i,i]=-1*sum(q_mat[i,j] for j in 1:5 if j!=i)
    end
    return [q_mat,eigvecs(q_mat),eigvals(q_mat)]
end

function get_IAM_factor(t)
    return 1
end

function give_sorted(t)
    v=sortperm(t)
    return v
end


give_sorted (generic function with 1 method)

In [6]:
q=zeros(5,5,nrow(innovations))
a=zeros(5,5,nrow(innovations))
ainv=zeros(5,5,nrow(innovations))
d=zeros(5,1,nrow(innovations))
s0=zeros(1,5,nrow(innovations))
for i in 1:nrow(innovations)
    q[:,:,i]=round.(get_rate_eig_mat(i)[1],digits=3)
    a[:,:,i]=(get_rate_eig_mat(i)[2])
    d[:,:,i]=round.(get_rate_eig_mat(i)[3],digits=3)
    ainv[:,:,i]=round.(inv(get_rate_eig_mat(i)[2]),digits=3)
    s0[1,rl[i],i]=1
end
rl_vals=[1 2 3 4 5]

1×5 Matrix{Int64}:
 1  2  3  4  5

In [7]:
inn_reg = CSV.File("./data/inn_reg.csv",header=1,delim=",") |> DataFrame
inn_xrang = CSV.File("./data/inn_xrang.csv",header=1,delim=",") |> DataFrame
inn_yrang = CSV.File("./data/inn_yrang.csv",header=1,delim=",") |> DataFrame
inn_yrang[!,"Y0"]=-1*(innovations[!,"RL"].-5)
inn_yrang

,Column1,Y0,Y1,Y2,Y3,Y4,Y5
,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,1,2,1.55336,1.08482,0.987212,1.26794,1.87998
2,2,3,3.55746,3.93607,4.40891,5.05482,5.76446
3,3,4,3.16098,2.53288,2.48238,2.96874,3.96047
4,4,2,1.90803,2.33811,3.25335,4.37439,5.72498
5,5,4,4.03011,4.32668,4.96027,5.88777,7.0267


In [8]:
function tech_roadmap_mccormick_peicewise_self()
    
    rdmap= Model(BARON.Optimizer)
    set_optimizer_attribute(rdmap, "MaxTime", 2500)
    
    #Sets
    # i: innovation
    # k: compromise or pareto optimal solution
    # r,c : row column of transition matrices
    
    #Declaration of variables
    
    #binary variable choosing the pareto optimal solution
    @variable(rdmap,yk[k=1:nrow(compromise)],Bin)
    #time at which a pareto optimal solution is chosen
    @variable(rdmap,0 ≤ tk[k=1:nrow(compromise)] ≤ 50)
    #percentage of functional unit that a pareto optimal solution takes up
    @variable(rdmap,0 ≤ xk[k=1:nrow(compromise)] ≤ 1)
    
    #binary variable choosing which innovation to invest in for adoption
    @variable(rdmap,yi[i=1:nrow(innovations)],Bin)
    #time at which invested is made and innovation made adoptible
    @variable(rdmap,0 ≤ ti[i=1:nrow(innovations)] ≤ 50)
    #readiness level state value of ith innovation
    @variable(rdmap, s[i=1:nrow(innovations)] )
    
    #CO2 at 2050 target
    @variable(rdmap, co2_2050 ≥ 0)
    
    #circularity at 2030 target
    @variable(rdmap, theta_2030 ≥ 0)
    #recycled content at 2030 target
    @variable(rdmap, recycl_2030 ≥ 0)
    #compostable content at 2030 target
    @variable(rdmap, comp_2030 ≥ 0)
    
    #integral approximator of CO2
    @variable(rdmap, co2_tot ≥ 0)
    
    #cost incured in investing to adopt
    @variable(rdmap, 0 ≤ cost_inn)
    
    
    #Constraints
    
    #Planning optimization constraints    
    #only one compromise solution chosen on each pareto front, and chosen only if investment made in ith front
    @constraint(rdmap, [i=1], sum(yk[k] for k=1:mapki[i,2])==yi[i])
    @constraint(rdmap, [i=2:nrow(mapki)], sum(yk[k] for k=mapki[i-1,2]:mapki[i,2])==yi[i])
    
    #one can pick a compromise solution from ith front to satisfy functional unit only after investment in ith innovation made
    @constraint(rdmap, [i=1], ti[i] ≤ sum(tk[k] for k=1:mapki[i,2]))
    @constraint(rdmap, [i=2:nrow(mapki)], ti[i] ≤ sum(tk[k] for k=mapki[i-1,2]:mapki[i,2]))
    
    #if innovation invested in, time has to be between 0 and fifty, otherwise 0
    @constraint(rdmap, [i=1:nrow(mapki)], 50*yi[i] ≥ ti[i])
    #if compromise solution chosen, time has to be between 0 and fifty, otherwise 0
    @constraint(rdmap, [k=1:nrow(compromise)], 50*yk[k] ≥ tk[k])
    
    #if innovation invested in, time has to be between 0 and fifty, otherwise 0
    #@constraint(rdmap, [i=1:nrow(mapki)], 50*yi[i] ≤ ti[i])
    #if compromise solution chosen, time has to be between 0 and fifty, otherwise 0
    #@constraint(rdmap, [k=1:nrow(compromise)], 50*yk[k] ≤ tk[k])
    
    
    #functional unit allocation to innovation iff compromise solution chosen
    @constraint(rdmap, [k=1:nrow(compromise)], yk[k] ≥ xk[k])
    
    #total fraction of functional unit satisfied by innovative compromise solutions <1
    @constraint(rdmap, sum(xk[k] for k in 1:nrow(compromise)) ≤ 1) 
    
    #co2 at 2050 target formulated based on distribution of functional unit to various pareto solutions from innovative and conventional fronts
    @constraint(rdmap, co2_2050==sum(xk[k]*compromise[k,3] for k in 1:nrow(compromise))
        + (1-sum(xk[k] for k in 1:nrow(compromise)))*initial[1,3])
    
    #tk < 30
    @variable(rdmap, tk_2030[k=1:nrow(compromise)], Bin)
    @constraint(rdmap, [k=1:nrow(compromise)], 50*tk_2030[k] ≥ 30 - tk[k])
    
    #circularity at 2030 target
    @constraint(rdmap, theta_2030==sum(xk[k]*compromise[k,4]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,4])
    
    #recyclable content at 2030 target
    @constraint(rdmap, recycl_2030==sum(xk[k]*compromise[k,5]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,5])
    
    #compostable at 2030 target
    @constraint(rdmap, comp_2030==sum(xk[k]*compromise[k,6]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,6])
    
    
    #Time evolution of innovation
   
    
    #Integral approximation of co2 emitted during the entire time horizon
    
    #mccormick envelopes reduces computational time by 3 fold (see previous formulation for actual bilinear integral approximation)
    @variable(rdmap,0 ≤ wk[k=1:nrow(compromise)])
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≥ 50*xk[k]+tk[k]-50)
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≤ 50*xk[k])
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≤ tk[k])
    
    @NLconstraint(rdmap, co2_tot == 50*initial[1,3]-sum((50-tk[k])*xk[k]*(initial[1,3]-compromise[k,3]) for k in 1:nrow(compromise)))
    #@constraint(rdmap, co2_tot == 50*initial[1,3]-sum((50*xk[k]-wk[k])*(initial[1,3]-compromise[k,3]) for k in 1:nrow(compromise)))

    #setting target for 2050 co2 pledge
    @constraint(rdmap, co2_2050 ≤ 44)
    
    #setting target for 2030 circularity pledge
    #@constraint(rdmap, theta_2030 ≥ 0.72)
    
    #setting target for 2030 recycl pledge
    @constraint(rdmap, recycl_2030 ≥ 0.3)
    
    #setting target for 2030 comp pledge
    @constraint(rdmap, comp_2030 ≥ 0.45)
    
    
    @variable(rdmap,f_fit[i=1:nrow(innovations)] ≥ 0)

    for i in 1:nrow(innovations)
        f_fit[i] = piecewiselinear(rdmap, ti[i], Matrix(inn_xrang[!,2:ncol(inn_xrang)])[i,:],Matrix(inn_yrang[!,2:ncol(inn_yrang)])[i,:])
    end

    
    @NLconstraint(rdmap, cost_inn == sum(ci[i]*(yi[i]*f_fit[i]) for i in 1:nrow(innovations)))
                

    
    @constraint(rdmap, [i=1:nrow(innovations)],s[i]==f_fit[i])
    
    #@variable(rdmap, y1, Bin)
    #@NLconstraint(rdmap, cost_inn == sum(ci[i]*(yi[i]*(y1*(2*t/50)+(1-y1)*(3*t/50))) for i in 1:nrow(innovations)))
    
    

    #definining objective function as investment cost + Co2 emission cost during the time horizon
    #40*0.001*co2_tot+
    #@NLconstraint(rdmap, obj_val == cost_inn)
    @NLobjective(rdmap,Min, 60*0.001*co2_tot+cost_inn)
    #supplying lower bound on objective based on cost_inn optimal
    #@variable(rdmap, obj_val ≥ 180)
    #print(rdmap)
    optimize!(rdmap)
    
    return JuMP.value.(yk), JuMP.value.(tk), JuMP.value.(xk), JuMP.value.(yi), JuMP.value.(ti), 
        JuMP.value.(s), JuMP.value.(co2_2050), JuMP.value.(cost_inn), JuMP.value.(co2_tot)
end




tech_roadmap_mccormick_peicewise_self (generic function with 1 method)

In [9]:
yk, tk, xk, yi, ti, s, co2, costInn, co2tot=tech_roadmap_mccormick_peicewise_self()
yi


 BARON version 21.1.13. Built: LNX-64 Wed Jan 13 16:09:31 EST 2021

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Bhavik R. Bakshi at The Ohio State University, bakshi.2@osu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Kilinc, M. and N. V. Sahinidis, Exploiting integrality in the global
 optimization of mixed-integer nonlinear programming problems in BARON,
 Optimization Methods and Software, 33, 540-562, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Doing local search
 Solving bounding LP
 Starting multi-start local search
 Preprocessing found feasible solution with value  715.398895709     
 Preprocessing found feasible solution with value  572.826542393     
 Done with local search
  Iteration    Open nodes         Time (s)    L

5-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 0.0

In [10]:
ti


5-element Vector{Float64}:
  0.0
  0.0
 16.250000000000004
  0.0
  0.0

In [11]:
costInn

413.56474994009506

In [12]:
tk

25-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 16.250000000000004
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [13]:
co2tot*40*0.001

103.67727272727268

In [14]:
yi

5-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 0.0

In [15]:
tk

25-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 16.250000000000004
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [16]:
xk


25-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.5454545454545451
 0.0
 0.0
 0.0
 0.454545454545455
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [17]:
yk

25-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
s

5-element Vector{Float64}:
 2.0
 3.0
 2.532881409488398
 2.0
 4.0

In [28]:
Matrix(iam_model[!,1:1])

3×1 Matrix{Int64}:
 2030
 2040
 2050

In [29]:
compromise

,k,i,GWP,Theta,Per_Recycl,Per_Comp,GHG_ElecInd,GHG_Trans
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,40.0,0.6,0.5,0.0,0.301222,0.162415
2,2,1,43.0,0.66,0.55,0.0,0.291677,0.145083
3,3,1,45.5,0.68,0.567,0.0,0.323371,0.161281
4,4,1,47.5,0.7,0.583,0.0,0.525025,0.151392
5,5,2,30.0,0.4,0.0,0.2,0.308217,0.0923318
6,6,2,32.0,0.48,0.0,0.24,0.419839,0.167281
7,7,2,34.0,0.5,0.0,0.27,0.418893,0.136562
8,8,2,38.0,0.52,0.0,0.34,0.530551,0.134121
9,9,2,40.0,0.54,0.0,0.37,0.418593,0.146208


In [40]:
Matrix(inn_xrang[!,2:ncol(inn_xrang)])[:,1]

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [51]:
function tech_roadmap_mccormick_peicewise_iam()
    
    rdmap= Model(BARON.Optimizer)
    set_optimizer_attribute(rdmap, "MaxTime", 2500)
    
    #Sets
    # i: innovation
    # k: compromise or pareto optimal solution
    # r,c : row column of transition matrices
    
    #Declaration of variables
    
    #binary variable choosing the pareto optimal solution
    @variable(rdmap,yk[k=1:nrow(compromise)],Bin)
    #time at which a pareto optimal solution is chosen
    @variable(rdmap,0 ≤ tk[k=1:nrow(compromise)] ≤ 50)
    #percentage of functional unit that a pareto optimal solution takes up
    @variable(rdmap,0 ≤ xk[k=1:nrow(compromise)] ≤ 1)
    
    #binary variable choosing which innovation to invest in for adoption
    @variable(rdmap,yi[i=1:nrow(innovations)],Bin)
    #time at which invested is made and innovation made adoptible
    @variable(rdmap,0 ≤ ti[i=1:nrow(innovations)] ≤ 50)
    #readiness level state value of ith innovation
    @variable(rdmap, s[i=1:nrow(innovations)] )
    
    #CO2 at 2050 target
    @variable(rdmap, co2_2050 ≥ 0)
    
    #circularity at 2030 target
    @variable(rdmap, theta_2030 ≥ 0)
    #recycled content at 2030 target
    @variable(rdmap, recycl_2030 ≥ 0)
    #compostable content at 2030 target
    @variable(rdmap, comp_2030 ≥ 0)
    
    #integral approximator of CO2
    @variable(rdmap, co2_tot ≥ 0)
    
    #cost incured in investing to adopt
    @variable(rdmap, 0 ≤ cost_inn)
    
    #surrogate models
    @variable(rdmap,f_fit[i=1:nrow(innovations)] ≥ 0)
    @variable(rdmap,co2_correct[k=1:nrow(compromise)] ≥ 0)

    for i in 1:nrow(innovations)
        f_fit[i] = piecewiselinear(rdmap, ti[i], Matrix(inn_xrang[!,2:ncol(inn_xrang)])[i,:],Matrix(inn_yrang[!,2:ncol(inn_yrang)])[i,:])
    end
    
    for k in 1:nrow(compromise)
        co2_correct[k] = compromise[k,3]*compromise[k,7]*(1-piecewiselinear(rdmap, tk[k], Matrix(iam_model[!,1:1])[:,1],Matrix(iam_model[!,2:2])[:,1]))
    end
    
    
    #Constraints
    
    
    #Planning optimization constraints    
    #only one compromise solution chosen on each pareto front, and chosen only if investment made in ith front
    @constraint(rdmap, [i=1], sum(yk[k] for k=1:mapki[i,2])==yi[i])
    @constraint(rdmap, [i=2:nrow(mapki)], sum(yk[k] for k=mapki[i-1,2]:mapki[i,2])==yi[i])
    
    #one can pick a compromise solution from ith front to satisfy functional unit only after investment in ith innovation made
    @constraint(rdmap, [i=1], ti[i] ≤ sum(tk[k] for k=1:mapki[i,2]))
    @constraint(rdmap, [i=2:nrow(mapki)], ti[i] ≤ sum(tk[k] for k=mapki[i-1,2]:mapki[i,2]))
    
    #if innovation invested in, time has to be between 0 and fifty, otherwise 0
    @constraint(rdmap, [i=1:nrow(mapki)], 50*yi[i] ≥ ti[i])
    #if compromise solution chosen, time has to be between 0 and fifty, otherwise 0
    @constraint(rdmap, [k=1:nrow(compromise)], 50*yk[k] ≥ tk[k])
    
    #if innovation invested in, time has to be between 0 and fifty, otherwise 0
    #@constraint(rdmap, [i=1:nrow(mapki)], 50*yi[i] ≤ ti[i])
    #if compromise solution chosen, time has to be between 0 and fifty, otherwise 0
    #@constraint(rdmap, [k=1:nrow(compromise)], 50*yk[k] ≤ tk[k])
    
    
    #functional unit allocation to innovation iff compromise solution chosen
    @constraint(rdmap, [k=1:nrow(compromise)], yk[k] ≥ xk[k])
    
    #total fraction of functional unit satisfied by innovative compromise solutions <1
    @constraint(rdmap, sum(xk[k] for k in 1:nrow(compromise)) ≤ 1) 
    
    #co2 at 2050 target formulated based on distribution of functional unit to various pareto solutions from innovative and conventional fronts
    @constraint(rdmap, co2_2050==sum(xk[k]*compromise[k,3] for k in 1:nrow(compromise))
        + (1-sum(xk[k] for k in 1:nrow(compromise)))*initial[1,3])
    
    #tk < 30
    @variable(rdmap, tk_2030[k=1:nrow(compromise)], Bin)
    @constraint(rdmap, [k=1:nrow(compromise)], 50*tk_2030[k] ≥ 30 - tk[k])
    
    #circularity at 2030 target
    @constraint(rdmap, theta_2030==sum(xk[k]*compromise[k,4]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,4])
    
    #recyclable content at 2030 target
    @constraint(rdmap, recycl_2030==sum(xk[k]*compromise[k,5]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,5])
    
    #compostable at 2030 target
    @constraint(rdmap, comp_2030==sum(xk[k]*compromise[k,6]*(tk_2030[k]) for k in 1:nrow(compromise))
        + (1-sum(xk[k]*(tk_2030[k]) for k in 1:nrow(compromise)))*initial[1,6])
    
    
    #Time evolution of innovation
   
    
    #Integral approximation of co2 emitted during the entire time horizon
    
    #mccormick envelopes reduces computational time by 3 fold (see previous formulation for actual bilinear integral approximation)
    @variable(rdmap,0 ≤ wk[k=1:nrow(compromise)])
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≥ 50*xk[k]+tk[k]-50)
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≤ 50*xk[k])
    @constraint(rdmap,[k=1:nrow(compromise)],wk[k] ≤ tk[k])
    
    @NLconstraint(rdmap, co2_tot == 50*initial[1,3]-sum((50-tk[k])*xk[k]*(initial[1,3]-compromise[k,3]) for k in 1:nrow(compromise)))
    #@constraint(rdmap, co2_tot == 50*initial[1,3]-sum((50*xk[k]-wk[k])*(initial[1,3]-compromise[k,3]) for k in 1:nrow(compromise)))

    #setting target for 2050 co2 pledge
    @constraint(rdmap, co2_2050 ≤ 44)
    
    #setting target for 2030 circularity pledge
    #@constraint(rdmap, theta_2030 ≥ 0.72)
    
    #setting target for 2030 recycl pledge
    @constraint(rdmap, recycl_2030 ≥ 0.3)
    
    #setting target for 2030 comp pledge
    @constraint(rdmap, comp_2030 ≥ 0.45)
    
    


    
    @NLconstraint(rdmap, cost_inn == sum(ci[i]*(yi[i]*f_fit[i]) for i in 1:nrow(innovations)))
                

    
    @constraint(rdmap, [i=1:nrow(innovations)],s[i]==f_fit[i])
    
    #@variable(rdmap, y1, Bin)
    #@NLconstraint(rdmap, cost_inn == sum(ci[i]*(yi[i]*(y1*(2*t/50)+(1-y1)*(3*t/50))) for i in 1:nrow(innovations)))
    
    

    #definining objective function as investment cost + Co2 emission cost during the time horizon
    #40*0.001*co2_tot+
    #@NLconstraint(rdmap, obj_val == cost_inn)
    @NLobjective(rdmap,Min, 60*0.001*co2_tot+cost_inn)
    #supplying lower bound on objective based on cost_inn optimal
    #@variable(rdmap, obj_val ≥ 180)
    #print(rdmap)
    optimize!(rdmap)
    
    return JuMP.value.(yk), JuMP.value.(tk), JuMP.value.(xk), JuMP.value.(yi), JuMP.value.(ti), 
        JuMP.value.(s), JuMP.value.(co2_2050), JuMP.value.(cost_inn), JuMP.value.(co2_tot)
end




tech_roadmap_mccormick_peicewise_iam (generic function with 1 method)

In [52]:
yk, tk, xk, yi, ti, s, co2, costInn, co2tot=tech_roadmap_mccormick_peicewise_iam()
yi


LoadError: MethodError: [0mCannot `convert` an object of type [92mAffExpr[39m[0m to an object of type [91mVariableRef[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::GenericQuadExpr{T, VarType} where VarType[39m) where T at /home/thakker.8/.julia/packages/JuMP/qhoVb/src/quad_expr.jl:328
[0m  convert(::Type{T}, [91m::GenericAffExpr{T, VarType} where VarType[39m) where T at /home/thakker.8/.julia/packages/JuMP/qhoVb/src/aff_expr.jl:317
[0m  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:205
[0m  ...